## Exploratory Data Analysis

Setup logging and progress bar

In [11]:
import logging, csv
import numpy as np
import decord
import torch
import os, time
import pandas as pd
import random

from gluoncv.torch.utils.model_utils import download
from gluoncv.torch.data.transforms.videotransforms import video_transforms, volume_transforms
from gluoncv.torch.engine.config import get_cfg_defaults
from gluoncv.torch.model_zoo import get_model
from tqdm.notebook import tqdm

# Configure the logging settings
logging.basicConfig(filename='eda-output.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

map_path = 'action_map.csv'

action_dict = {}

with open(map_path, 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        id_value = int(row['id'])
        name_value = row['name']
        
        action_dict[id_value] = name_value

Load dataset

In [12]:
def process_frames(file_path, model, curr_class, N):
    vr = decord.VideoReader(file_path)
    new_rows = []
    for i in range(N, len(vr) - N, (len(vr) - 2*N - 1) // 2):
        frame_id_list = range(i-N, i+N+1)
        video_data = vr.get_batch(frame_id_list).asnumpy()
        crop_size = 224
        short_side_size = 256
        transform_fn = video_transforms.Compose([video_transforms.Resize(short_side_size, interpolation='bilinear'),
                                                video_transforms.CenterCrop(size=(crop_size, crop_size)),
                                                volume_transforms.ClipToTensor(),
                                                video_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        clip_input = transform_fn(video_data)
        with torch.no_grad():
            pred = model(torch.unsqueeze(clip_input, dim=0)).numpy()
        probs = torch.nn.functional.softmax(torch.tensor(pred), dim=1).numpy()
        top_class = np.argmax(probs)
        confidence_interval = np.max(probs) - np.min(probs)

        position_str = "S"
        if i == N + ((len(vr) - 2*N - 1) // 2):
            position_str = "M"
        elif i == N + 2 * ((len(vr) - 2*N - 1) // 2):
            position_str = "E"
        new_row = {"file_name": file_path.split("/")[-1], "class_id": curr_class, "class_name": action_dict[curr_class], "pred_class_id": top_class, "pred_class_name": action_dict[top_class], "window_size": N*2+1, "confidence": confidence_interval, "is_pred_correct": True if top_class == curr_class else False, "position": position_str}
        new_rows.append(new_row)
    return new_rows

In [14]:
dataset_path = 'datasets/kinetics400_5per/train'

subfolders = sorted([subfolder for subfolder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, subfolder))])
random_subfolders = random.sample(subfolders, 50)
subfolder = random_subfolders[0]

config_file = './i3d_resnet50_v1_kinetics400.yaml'
cfg = get_cfg_defaults()
cfg.merge_from_file(config_file)
model = get_model(cfg)
model.eval()

columns = ["file_name", "class_id", "class_name", "pred_class_id", "pred_class_name", "window_size", "confidence", "is_pred_correct", "position"]

my_data = []

try:
    # Iterate over subfolders in the dataset path
    for index, subfolder in tqdm(enumerate(random_subfolders), desc=f"Folder progress", position=0, total=len(random_subfolders)):
        subfolder_path = os.path.join(dataset_path, subfolder)
        files = os.listdir(subfolder_path)
        clean_files = []
        for filename in files:
            if filename.endswith(".mp4"):
                clean_files.append(filename)
        random_files = random.sample(clean_files, 3)
    
        # Iterate over files in the subfolder
        for filename in tqdm(random_files, desc=f"File progress for folder: {subfolder}", position=1, leave=False):
            file_path = os.path.join(subfolder_path, filename)

            N = 2
            # Do something with each file
            for N in tqdm(range(N, 10), desc=f"Context window progress for file: {filename}", position=2, leave=False):
                new_rows = process_frames(file_path, model, index, N)
                for row in new_rows:
                    my_data.append(row)
except Exception as e:
    print(e)

finally:
    # Specify the file path
    out_path = 'eda_outputs.csv'
    
    # Check if the file exists
    if os.path.exists(out_path):
        # Delete the file
        os.remove(out_path)
    df = pd.DataFrame(my_data, columns=columns)
    df.to_csv(out_path, mode='w', index=False)

Folder progress:   0%|          | 0/50 [00:00<?, ?it/s]

File progress for folder: opening present:   0%|          | 0/3 [00:00<?, ?it/s]

Context window progress for file: vaIdBL8WgXE.mp4:   0%|          | 0/8 [00:00<?, ?it/s]

Context window progress for file: mRb3CmgbjBo.mp4:   0%|          | 0/8 [00:00<?, ?it/s]

Context window progress for file: XF-jJ2RCpYo.mp4:   0%|          | 0/8 [00:00<?, ?it/s]

...

Context window progress for file: ueR-yY8QWmY.mp4:   0%|          | 0/8 [00:00<?, ?it/s]

File progress for folder: playing trombone:   0%|          | 0/3 [00:00<?, ?it/s]

Context window progress for file: L6ZahNKQXpk.mp4:   0%|          | 0/8 [00:00<?, ?it/s]

Context window progress for file: Vh-XSIyJ5dA.mp4:   0%|          | 0/8 [00:00<?, ?it/s]

Context window progress for file: qoUIB149rgU.mp4:   0%|          | 0/8 [00:00<?, ?it/s]